In [408]:
from generator import *
from shredder import *
from fitter import *
from exec_tools import run
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from joblib import Parallel, delayed
from math import sqrt
import time
import itertools
#np.random.seed(10)



In [409]:
data_gen_dict = config_maker(2, 10, [.5], [[1,2]],[5])
data_gen_dict = config_maker(4, 10, [1,1,1], [],[])

In [ ]:
estimates = list()
in_ci = list()
for iters in range(10000):
    fit_data, test_data, true_coeffs = generate_ind_model(
                                            data_gen_dict["dist_list"],
                                            data_gen_dict["main_coeffs"],
                                            data_gen_dict["interaction_coeffs"],
                                            data_gen_dict["interactions"],
                                            intercept=10,
                                            n=100,
                                            beta_sigma=.1
                                        )

    tmp_data = fit_data.copy(deep=True)
#     tmp_data = uniform_shred_cols(['x1'], .01, tmp_data)
#     tmp_data = tmp_data.dropna()
    model, metrics = fit_lm(tmp_data, test_data)
    b_estimate_results = beta_target_check(metrics, true_coeffs, as_dataframe=False)
    in_ci.append([b_estimate_results[key]['in_ci'] for key in b_estimate_results.keys()])
    
    estimates.append(model.params.values)

estimates_df = pd.DataFrame(estimates, columns=model.params.index)
in_ci_df = pd.DataFrame(in_ci)

In [523]:
print(estimates_df.mean())
print(pd.DataFrame(in_ci_df.mean().values, columns=['% on target'], index=estimates_df.mean().index))

const    10.001262
x1        5.004447
x2        1.987765
x1:x2     3.998084
dtype: float64
       % on target
const        0.956
x1           0.957
x2           0.956
x1:x2        0.947


In [5]:
tmp_data = fit_data.copy(deep=True)
tmp_data = uniform_shred_cols(['x1'], 0, tmp_data)

test = pd.DataFrame(np.array([tmp_data['x1'], fit_data['x1']]).T, columns=['x1_altered', 'x1_original'])
print(test)
test[['x1_altered', 'x1_original']].assign(NE=test.x1_altered == test.x1_original)

    x1_altered  x1_original
0     0.522054     0.522054
1     0.879428     0.879428
2     0.762485     0.762485
3     0.864024     0.864024
4     0.722210     0.722210
5     0.531083     0.531083
6     0.613244     0.613244
7     0.629536     0.629536
8     0.959190     0.959190
9     0.177162     0.177162
10    0.597171     0.597171
11    0.614091     0.614091
12    0.732349     0.732349
13    0.893135     0.893135
14    0.475235     0.475235
15    0.255864     0.255864
16    0.674234     0.674234
17    0.096581     0.096581
18    0.210147     0.210147
19    0.186248     0.186248
20    0.340971     0.340971
21    0.683949     0.683949
22    0.424472     0.424472
23    0.508685     0.508685
24    0.511914     0.511914
25    0.715462     0.715462
26    0.874424     0.874424
27    0.123111     0.123111
28    0.412546     0.412546
29    0.965554     0.965554
..         ...          ...
70    0.839825     0.839825
71    0.585306     0.585306
72    0.650935     0.650935
73    0.255413     0

,x1_altered,x1_original,NE
0,0.522054,0.522054,True
1,0.879428,0.879428,True
2,0.762485,0.762485,True
3,0.864024,0.864024,True
4,0.722210,0.722210,True
5,0.531083,0.531083,True
6,0.613244,0.613244,True
7,0.629536,0.629536,True
8,0.959190,0.959190,True
9,0.177162,0.177162,True


In [6]:
print(true_coeffs)
print(model.summary())


{'x1': 10, 'x2': 10, 'x3': 10, 'x4': 10, 'const': 10}
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 9.403e+30
Date:                Fri, 02 Mar 2018   Prob (F-statistic):               0.00
Time:                        14:56:06   Log-Likelihood:                 1576.0
No. Observations:                  50   AIC:                            -3142.
Df Residuals:                      45   BIC:                            -3133.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

In [7]:
model.params.values


array([ 10.,  10.,  10.,  10.,  10.])

# Rewrite of the Invert 
It seems that it still only works to invert if only one column is missing. If more than one is missing then you're still out of luck. 

In [508]:
#data_gen_dict = config_maker(3, 5, [.4, .4], [[1,2], [1,3], [2,3]],[5, 10, 8])
data_gen_dict = config_maker(
    2,
    5,
    [.4],
    [
        [1,2]
    ], 
    [4])
#data_gen_dict = config_maker(1, 10, [], [],[])
fit_data, test_data, true_coeffs = generate_ind_model(
                                        data_gen_dict["dist_list"],
                                        data_gen_dict["main_coeffs"],
                                        data_gen_dict["interaction_coeffs"],
                                        data_gen_dict["interactions"],
                                        intercept=1,
                                        n=100,
                                        beta_sigma=.2
                                    )

# Drop some data
#tmp_data = uniform_shred_cols(['x2'], .5, fit_data)
tmp_data = fit_data.copy()
tmp_data = uniform_shred_cols(['x2'], .4, tmp_data)
tmp_data = uniform_shred_cols(['x1'], .1, tmp_data)


import importlib
import fixer
importlib.reload(fixer)

inverted,  fitted = fixer.olsinv_singlex(tmp_data, 'x2', test_mode=False)

print(tmp_data.head(10))
print(inverted.head(10))

tmp_data.loc[inverted.index, 'x2'] = inverted

inverted,  fitted = fixer.olsinv_singlex(tmp_data, 'x1', test_mode=False)

print(tmp_data.head(10))
print(inverted.head(10))

tmp_data.loc[inverted.index, 'x1'] = inverted
print(tmp_data.head(40))

         x1        x2     x1:x2          y
0  0.380700       NaN  0.154304   5.621569
1  0.416614       NaN  0.397601   7.147583
2  0.864448  0.230721  0.199446   7.069593
3  0.206235  0.862755  0.177931   3.867554
4  0.786126  0.582451  0.457879   8.024719
5  0.228217  0.581049  0.132605   3.949757
6  0.767155       NaN  0.688854  10.206997
7  0.747678       NaN  0.667406   8.046303
8  0.594156  0.948097  0.563318   8.083091
9  0.995979       NaN  0.727283  12.431498
0     0.759323
1     1.079644
6     1.030385
7     0.647650
9     1.047196
11    0.031282
15    0.579573
23    1.229719
24    0.854970
27    0.203789
Name: y, dtype: float64
         x1        x2     x1:x2          y
0  0.380700  0.759323  0.154304   5.621569
1  0.416614  1.079644  0.397601   7.147583
2  0.864448  0.230721  0.199446   7.069593
3  0.206235  0.862755  0.177931   3.867554
4  0.786126  0.582451  0.457879   8.024719
5  0.228217  0.581049  0.132605   3.949757
6  0.767155  1.030385  0.688854  10.206997
7  0.7476